In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import nlopt
import molpack as mpk
import molpy as mp


In [2]:
def myfunc(x, grad):
    if grad.size > 0:
        grad[0] = 0.0
        grad[1] = 0.5 / np.sqrt(x[1])
    return np.sqrt(x[1])


def myconstraint1(x, grad):
    a = 2
    b = 0
    if grad.size > 0:
        grad[0] = 3 * a * (a*x[0] + b) * (a*x[0] + b)
        grad[1] = -1.0
    return ((a*x[0] + b) ** 3 - x[1])

def myconstraint2(x, grad):
    a = -1
    b = 1
    if grad.size > 0:
        grad[0] = 3 * a * (a*x[0] + b) * (a*x[0] + b)
        grad[1] = -1.0
    return ((a*x[0] + b) ** 3 - x[1])

In [ ]:
opt = nlopt.opt(nlopt.LD_MMA, 2)
opt.set_min_objective(myfunc)
opt.add_inequality_constraint(myconstraint1, 1e-8)
opt.add_inequality_constraint(myconstraint2, 1e-8)
opt.set_lower_bounds(np.array([-np.inf, 0.0]))
opt.set_xtol_rel(1e-4)
opt.optimize(np.array([1.234, 5.678]))

array([0.33333333, 0.29629629])

In [ ]:
def write_xyz_traj(filename):
    """Write a trajectory to an xyz file."""
    f = open(filename, "w")

    def writer(points):
        f.write(f"{len(points)}\n")
        f.write("Comment\n")
        for point in points:
            f.write(f"X {point[0]} {point[1]} {point[2]}\n")

    return writer


In [5]:


writer = write_xyz_traj("output.xyz")

# === Objective wrapper ===
class ObjectiveFunction:
    def __init__(self, constraint):
        self.constraint = constraint

    def value(self, x):
        points = x.reshape(-1, 3)
        writer(points)
        penalty = self.constraint.penalty(points)
        return penalty

    def gradient(self, x):
        points = x.reshape(-1, 3)
        grad = self.constraint.dpenalty(points)
        return grad
    
    def __call__(self, x, grad):
        if grad.size > 0:
            grad[:] = self.gradient(x)

        return self.value(x)
    
class SingleWellConstraint:

    def __init__(self, center, offset):
        self.center = center
        self.offset = offset

    def penalty(self, x):
        points = x.reshape(-1, 3)
        penalty = (points - self.center) ** 2 - self.offset
        penalty = np.sum(penalty)
        return penalty
    
    def dpenalty(self, x):
        points = x.reshape(-1, 3)
        grad = 2 * (points - self.center)
        return grad.flatten()

# === Simulation parameters ===
N = 100
box_origin = np.array([0, 0, 0])
box_size = np.array([10, 10, 10])

# === Initialize system ===
np.random.seed(42)
x0 = np.random.uniform(low=0, high=10, size=(N, 3)).astype(np.float64).flatten()

constraints = SingleWellConstraint(center=np.array([5, 5, 5]), offset=1.0)

# === Optimize
obj = ObjectiveFunction(constraints)
opt = nlopt.opt(nlopt.LD_MMA, 3 * N)
opt.set_min_objective(obj)
opt.set_lower_bounds(np.array([0.0] * (3 * N)))
opt.set_upper_bounds(np.array([10.0] * (3 * N)))
opt.set_xtol_rel(1e-4)
opt.set_maxeval(1000)
result = opt.optimize(x0)
print("Optimized result:", result)


Optimized result: [5.0000053  5.0000053  4.9999947  4.99999468 5.00000547 5.00000547
 4.9999947  4.99999458 4.99999469 4.9999947  4.99999464 5.00000532
 4.99999453 5.0000054  5.00000546 5.00000546 5.0000053  4.9999947
 5.00000536 5.0000053  4.9999947  5.00000544 5.0000053  5.0000053
 5.00000533 4.99999461 5.00000543 4.9999947  4.99999467 4.9999947
 4.99999469 5.00000547 4.99999458 5.0000053  5.00000531 4.99999455
 5.0000053  5.0000053  4.9999947  5.00000536 5.00000537 5.0000054
 4.99999469 4.9999947  5.00000531 4.9999947  5.0000053  4.9999947
 4.99999467 5.00000546 5.00000532 4.99999463 5.0000053  4.9999947
 4.99999468 4.99999466 5.00000531 5.00000544 4.9999947  5.0000053
 5.0000053  5.0000053  4.99999453 5.0000053  5.0000053  4.99999468
 5.00000544 4.99999457 5.00000556 5.0000054  4.99999464 5.00000543
 4.99999457 4.99999454 4.9999947  4.9999947  4.99999464 5.0000056
 5.0000053  5.00000534 4.99999457 4.9999947  5.00000531 4.99999466
 5.0000053  5.0000053  4.9999947  4.9999947  4.99999

In [ ]:

writer = write_xyz_traj("output.xyz")


# === Objective wrapper ===
class ObjectiveFunction:
    def __init__(self, constraint):
        self.constraint = constraint

    def value(self, x):
        points = x.reshape(-1, 3)
        writer(points)
        penalty = self.constraint.penalty(points)
        return penalty

    def gradient(self, x):
        points = x.reshape(-1, 3)
        grad = self.constraint.dpenalty(points)
        return grad

    def __call__(self, x, grad):
        if grad.size > 0:
            grad[:] = self.gradient(x).flatten()

        return self.value(x)


# === Simulation parameters ===

N = 100
box_origin = np.array([0, 0, 0])
box_size = np.array([10, 10, 10])

# === Initialize system ===
np.random.seed(42)
x0 = np.random.uniform(low=0, high=10, size=(N, 3)).astype(np.float64).flatten()

constraints = mpk.MinDistanceConstraint(1.0) & ~mpk.InsideSphereConstraint(
    center=np.array([5, 5, 5]), radius=5.0
)

# === Optimize
obj = ObjectiveFunction(constraints)
opt = nlopt.opt(nlopt.LD_MMA, 3 * N)
opt.set_min_objective(obj)
opt.set_lower_bounds(np.array([0.0] * (3 * N)))
opt.set_upper_bounds(np.array([10.0] * (3 * N)))
opt.set_xtol_rel(1e-4)
opt.set_maxeval(1000)
result = opt.optimize(x0)
isin = mpk.InsideSphereConstraint(
    center=np.array([5, 5, 5]), radius=5.0
).region.isin(result.reshape(-1, 3))
assert isin.sum() == 0, f"{isin.sum()} points are inside the constraint region."

In [2]:
import molpy as mp
tip3p = mp.builder.water.tip3p
region = mp.region.BoxRegion([10, 10, 10], [0, 0, 0])
builder = mp.builder.rand.UniformRandomBuilder(region)
structs = builder.fill(30, tip3p)
frame = mp.Frame.from_structs(structs)

In [3]:
with mp.io.data.XYZWriter("init.xyz") as writer:
    writer.write(frame)

In [ ]:
import molpy.pack as mpk

N = 100
box_origin = np.array([0, 0, 0])
box_size = np.array([10, 10, 10])
sphere_center = np.array([5, 5, 5])
sphere_radius = 3.0

# === Initialize system ===
np.random.seed(42)
x0 = np.random.uniform(low=0, high=10, size=(N, 3)).astype(np.float64).flatten()

bounds = [(0, 10)] * (3 * N)

constraints = mpk.InsideBoxConstraint(
    box_size, box_origin
) & ~mpk.InsideSphereConstraint(sphere_radius, sphere_center)


app = mpk.Session("nlopt")
app.add_target(tip3p.to_frame(), 30, constraints)
frame = app.optimize()
with mp.io.data.XYZWriter("output.xyz") as writer:
    writer.write(frame)

0  O UNK A 1  0.0 0.0 0.0 X
1  H UNK A 1  0.0 0.239987 0.92705 X
2  H UNK A 1  0.0 0.9572 -0.24 X
